# Code for gathering gathering dataset
First cell) Here our MP4s will be converted to a large swath of images for labeling. 
Second cell) Here we will randomly sample from directories to create training dataset.

In [ ]:
import cv2
import os

def extract_frames_from_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Get a list of all MP4 files in the input folder
    video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]
    if not video_files:
        print("No MP4 files found in the input folder.")
        return
    
    print(f"Found {len(video_files)} MP4 files to process.")

    for video_file in video_files:
        video_path = os.path.join(input_folder, video_file)
        video_name = os.path.splitext(video_file)[0]
        
        # Create a subfolder for frames from this video
        video_output_folder = os.path.join(output_folder, video_name)
        if not os.path.exists(video_output_folder):
            os.makedirs(video_output_folder)
        
        # Open the video file
        video = cv2.VideoCapture(video_path)
        if not video.isOpened():
            print(f"Error: Could not open video {video_file}. Skipping...")
            continue
        
        # Extract frames
        frame_count = 0
        success, frame = video.read()
        print(f"Processing {video_file}...")
        
        while success:
            frame_filename = os.path.join(video_output_folder, f"frame_{frame_count:05d}.jpg")
            cv2.imwrite(frame_filename, frame)
            frame_count += 1
            
            # Read next frame
            success, frame = video.read()
            
            if frame_count % 100 == 0:
                print(f"Extracted {frame_count} frames from {video_file}...")
        
        video.release()
        print(f"Finished extracting {frame_count} frames from {video_file}.")

    print(f"All videos processed. Frames saved to {output_folder}.")

input_folder = "/Users/jakeziegler/Desktop/PROJECTS/SPRAGUE EYE/JSE TRAINING" 
output_folder = "frames_output" 
extract_frames_from_folder(input_folder, output_folder)


In [8]:
import pandas as pd
import os

def create_image_dataframe(root_directory):
    directories = []
    image_paths = []

    for subdir, _, files in os.walk(root_directory):
        for file in files:
            # Only process image files
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                full_path = os.path.join(subdir, file)
                directories.append(subdir.split("/")[-1])
                image_paths.append(full_path)
    
    df = pd.DataFrame({
        'Directory': directories,
        'Image Path': image_paths
    })
    return df

root_dir = "/Users/jakeziegler/Desktop/PROJECTS/SPRAGUE EYE/JSE TRAINING/FRAMES" 
image_df = create_image_dataframe(root_dir)
sample_df = image_df.sample(n=500, random_state=42)

file_paths = sample_df['Image Path']

# Destination folder
destination_folder = "/Users/jakeziegler/Desktop/PROJECTS/SPRAGUE EYE/JSE TRAINING/SAMPLED FRAMES"
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Loop through the Series and copy each file
for file_path in file_paths:
    if os.path.exists(file_path):
        shutil.copy(file_path, destination_folder)
        print(f"Copied: {file_path} to {destination_folder}")
    else:
        print(f"File not found: {file_path}")
